# Playstore Data Analysis

In [ ]:
from pyspark.sql import SparkSession
#from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName("Play Store Analysis").getOrCreate()

In [ ]:
df = spark.read.load('/FileStore/tables/googleplaystore.csv', format = 'csv', header = 'True', sep = ',', escape='"', inferschema= True)

In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [ ]:
df.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

## Data Cleaning
- Deleting unwanted column
- Correcting the Schema (DataTypes)


In [ ]:
# Dropping columns which are not as important
df = df.drop('Size','Content Rating','Last Updated','Current Ver','Android Ver')

In [ ]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [ ]:
# Dropping duplicates
df = df.dropDuplicates()

In [ ]:
# There are some same app with minute difference we will also treat them as duplicate
df.filter(df.App == 'Minecraft').show()

+---------+--------+------+-------+-----------+----+-----+--------------------+
|      App|Category|Rating|Reviews|   Installs|Type|Price|              Genres|
+---------+--------+------+-------+-----------+----+-----+--------------------+
|Minecraft|  FAMILY|   4.5|2375336|10,000,000+|Paid|$6.99|Arcade;Action & A...|
|Minecraft|  FAMILY|   4.5|2376564|10,000,000+|Paid|$6.99|Arcade;Action & A...|
+---------+--------+------+-------+-----------+----+-----+--------------------+



In [ ]:
df = df.dropDuplicates(["App"])


In [ ]:
# Remove '+' ',' and '$' and cast the result to an integer 
df = df.withColumn('Installs', regexp_replace(col('Installs'), '[+,]', '').cast('int'))\
       .withColumn('Price', regexp_replace(col('Price'), '[$]', '').cast('int'))\
       .withColumn('Rating', col('Rating').cast('int'))\
       .withColumn('Reviews', col('Reviews').cast('int'))


In [ ]:
df.show(2)

+--------------------+--------+------+-------+--------+----+-----+------+
|                 App|Category|Rating|Reviews|Installs|Type|Price|Genres|
+--------------------+--------+------+-------+--------+----+-----+------+
|"i DT" Fútbol. To...|  SPORTS|     0|     27|     500|Free|    0|Sports|
|+Download 4 Insta...|  SOCIAL|     4|  40467| 1000000|Free|    0|Social|
+--------------------+--------+------+-------+--------+----+-----+------+
only showing top 2 rows



In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [ ]:
df.select("Type").distinct().show()

+----+
|Type|
+----+
|   0|
| NaN|
|Free|
|Paid|
+----+



In [ ]:
# Replace 0 and NaN with Free

df = df.withColumn("Type", when(col("Type") == 'NaN', "Free")
    .when(col("Type") == 0, "Free")     
    .otherwise(col("Type")))


In [ ]:
df.select("Type").distinct().show()

+----+
|Type|
+----+
|Free|
|Paid|
+----+



## SQL 

In [ ]:
# Create a view for using SQL Commands
df.createOrReplaceTempView('apps')

In [ ]:
%sql SELECT * FROM apps

App,Category,Rating,Reviews,Installs,Type,Price,Genres
"""i DT"" Fútbol. Todos Somos Técnicos.",SPORTS,0,27,500,Free,0,Sports
+Download 4 Instagram Twitter,SOCIAL,4,40467,1000000,Free,0,Social
- Free Comics - Comic Apps,COMICS,3,115,10000,Free,0,Comics
.R,TOOLS,4,259,10000,Free,0,Tools
/u/app,COMMUNICATION,4,573,10000,Free,0,Communication
058.ba,NEWS_AND_MAGAZINES,4,27,100,Free,0,News & Magazines
1. FC Köln App,SPORTS,4,2019,100000,Free,0,Sports
10 Best Foods for You,HEALTH_AND_FITNESS,4,2490,500000,Free,0,Health & Fitness
10 Minutes a Day Times Tables,FAMILY,4,681,100000,Free,0,Education
10 WPM Amateur ham radio CW Morse code trainer,COMMUNICATION,3,10,100,Paid,1,Communication


## Top Categories by Number of Installs:
**Question:** Which categories have the highest number of installations?

In [ ]:
%sql
SELECT Category, SUM(Installs) AS Total_Installs
FROM apps
GROUP BY Category
ORDER BY Total_Installs DESC;


Category,Total_Installs
GAME,13878924415
COMMUNICATION,11038276251
TOOLS,8001771915
PRODUCTIVITY,5793091369
SOCIAL,5487867902
PHOTOGRAPHY,4649147655
FAMILY,4427941505
VIDEO_PLAYERS,3926902720
TRAVEL_AND_LOCAL,2894887146
NEWS_AND_MAGAZINES,2369217760


## Top installed app and distribution of type
**Question:** What are the top installed app and their type (Free/Paid)?

In [ ]:
%sql
SELECT App, sum(Installs) as Total_Installs, Type
FROM apps
GROUP BY App , Type
ORDER BY Total_Installs DESC
LIMIT 30;

App,Total_Installs,Type
Facebook,1000000000,Free
Gmail,1000000000,Free
Google,1000000000,Free
Google Chrome: Fast & Secure,1000000000,Free
Google Drive,1000000000,Free
Google News,1000000000,Free
Google Photos,1000000000,Free
Google Play Books,1000000000,Free
Google Play Games,1000000000,Free
Google Play Movies & TV,1000000000,Free


## Impact of Reviews on Ratings

**Question:** How do the number of reviews affect the average rating of apps?


In [ ]:
%sql
SELECT Reviews, AVG(Rating) AS Avg_Rating
FROM apps
GROUP BY Reviews
ORDER BY Reviews DESC;


Reviews,Avg_Rating
78158306,4.0
69119316,4.0
66577313,4.0
56642847,4.0
44891723,4.0
42916526,4.0
27722264,4.0
25655305,4.0
24900999,4.0
23133508,4.0


## Pricing Strategy Analysis

**Question:** What is the relationship between app price and the number of installations?

In [ ]:
%sql
SELECT Round(AVG(Installs)) AS Avg_Installs, Price
FROM apps
WHERE Type = 'Paid'
GROUP BY Price
ORDER BY Avg_Installs DESC;


Avg_Installs,Price
695828.0,6
133706.0,0
87033.0,5
76332.0,2
53333.0,11
50000.0,13
45447.0,4
36700.0,24
32149.0,1
29230.0,9


## Genre Distribution Analysis

**Question:** What are the most common genres among the highest-rated apps?


In [ ]:
%sql
SELECT Genres, AVG(Rating) AS Avg_Rating
FROM apps
WHERE Type = 'Paid'
GROUP BY Genres
ORDER BY Avg_Rating DESC;


Genres,Avg_Rating
Adventure;Action & Adventure,4.0
Education;Pretend Play,4.0
Simulation;Pretend Play,4.0
Adventure,4.0
Food & Drink,4.0
Sports;Action & Adventure,4.0
Casual;Pretend Play,4.0
Education;Action & Adventure,4.0
Action;Action & Adventure,4.0
Board;Action & Adventure,4.0


## Free vs Paid App Performance

**Question:** How do free apps compare to paid apps in terms of average installations?


In [ ]:
%sql
SELECT Type, ROUND(AVG(Installs),2) AS Avg_Installs
FROM apps
GROUP BY Type;


Type,Avg_Installs
Free,8431491.93
Paid,75879.47


## Revenue Potential of Paid Apps

**Question:** Which paid apps have the highest revenue potential (considering price and installs)?

In [ ]:
%sql
SELECT App, Price, Installs, (Price * Installs) AS Revenue
FROM apps
WHERE Type = 'Paid'
ORDER BY Revenue DESC;

App,Price,Installs,Revenue
Minecraft,6,10000000,60000000
I am rich,399,100000,39900000
I Am Rich Premium,399,50000,19950000
Grand Theft Auto: San Andreas,6,1000000,6000000
Facetune - For Free,5,1000000,5000000
Sleep as Android Unlock,5,1000000,5000000
DraStic DS Emulator,4,1000000,4000000
I'm Rich - Trump Edition,400,10000,4000000
I am Rich Plus,399,10000,3990000
💎 I'm rich,399,10000,3990000
